 # Signal Spaces
 
 ```{epigraph}
Programming starts with types and functions. You probably have some preconceptions
about what types and functions are: get rid of them! They will cloud your mind. 
 
 --Bartosz Milewski
 ```

Signal processing starts with a time series, we apply algorithms on the time series to analyze it. To systemetically organize such processing procedure, we need to model the process.

The signal spaces we are considering is finite discrete spaces whereas the time domain is the  set of natural numbers. The intepretation of this finite set of time axis is not within our model. In this setting, the signal spaces we are considering is simply $\mathbb C^N$, where $N$ is a natural number corresponding to the length of the signal.

We have two classes of time series analysis algorithms, the global analysis and localized analysis. The algebraic data structure (ADS) for the original signal is just a list of numbers, we name it 0-sig. 

For localized analysis we have two cases, the block transform and smooth block transform.

The block transform basically cut the original signal into serveral sub-signals with the same length and use the global algorithm performing on each sub-signal. This leads to the notion of 1-sig which is list of list of numbers or list of 0-sig.

The smooth version of block transform requires extensions on two end of each sub-signal, we use 3-sig to denote its algebraic data structure, which is a list of triple 0-sig, corresponding to the left extension, the sub-signal itself and the the right extension repectively.

For global analysis, we view it as a special case of 1-sig, which contains one object. This object's ADS is 0-sig representing the original signal. In other words, global analysis is special case of local analysis with one block.

To summerize, the following is our model:
1. Time domain is digitalize into natural number.
2. $\mathbb C^N$ is the Signal spaces.
3. Three algebraic data structure 0-sig, 1-sig and 3-sig corresponding to global transforms, block transforms and smooth block transform repectively. A particular analysis (or algoritm) is performed through the functor associated with the algebraic data structure.
4. $N=2^L, L,N\in\mathbb N$, preprocessing and time domain intepretation are outside the model.

In mathematical terms, we have defined three underlying signal categories: $\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K$ and $\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$. There are three classes of morphisms we are interested in:
1. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F}_i,\ \textrm{for}\ i=0,1,3$, this class consists of morphisms that are map from $\mathfrak{F}_i$ to itself. We call these morphisms closed.
2. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F}_j,\ \textrm{for}\ i,j=0,1,3,i\neq j$, these are the morphisms that go from one of the three signal categories to another one within the three.
3. $f:\mathfrak{F}_i\longrightarrow\mathfrak{F},\ \textrm{for}\ i=0,1,3$, morphisms from within goes outside of the signal categories.

Most our works on algoriths is to build the functions that are the maps between $\mathbb{C}^M$, that is to say for an algorithm $a$, $a:\mathbb{C}^M\longrightarrow\mathbb{C}^M$. The main patterns of the egineering work are providing systematical way to promote a given algorithm $a$ to morphism $f$, such technique is call lifting.

So, the engineering tasks are two folds, managing morphisms, providing patterns and tools that are used to lift functions on objects in each category to the morphism in certain class described above. In this way, the algorithm development can be focusing on the developing functions on objects of the categories. This is the main theme for our project.

Fortunately, the lifting schemes are quite mature, our efforts mostly are not developing them, rather are to learn how to use them effectively.

## Signal categories and localization
As described earler, we categorize our signal spaces into three categories: 0-sig, 1-sig, 3-sig, corresponding to type `[a]`, `[[a]]` and `[[TripleBlocks a]]`, where `a` is a type of `SigType`. We use type synonym `Sig_0`, `Sig_1` and `Sig_3` repectively. They all have  build-in functor, Applicative, Monad mong other many behaviors as we use list as container. `SigType` can be set initially to some numerical type through type synonym.

In [1]:
import Data.Bits

ispower2::Int->Bool
ispower2 n
    | n<=0 = False
    | otherwise = (n .&. (n-1))==0

### 0-sig

In [2]:
type SigType a = (Num a, Fractional a, Ord a, Show a)
type Sig_0 a = SigType a=>[a]

In [3]:
:k Sig_0

Sig_0 :: * -> *

### 1-sig
We use type synonym here. The other method is to use `newtype`. The advantage of using `newtype` is making the code more regulated as one needs to use the type name to construct the `Sig_1` type object  (the same goes to the other types). The price we pay for such regulation is the need to derive almost all useful type classes like `Functor`, `Show`, etc. Worst of all, there are quite a few `prefix` and `infix` operators, such as `take`, `(++)`, we need to reimplement. Although it is not a lot of work to redo all of these, for the moment, we stay with synonym.

`Sig_0`, `Sig_1` and `Sig_3` enforce the underline data type to be `SigType`, but this enforcement is not restricted, it works only if we code within the space of `Sig_0`, `Sig_1` and `Sig_3`. In many cases one can code with `[]`, as the `Sig` system is just the synonym of `[]`. This is the big drawback for using synonym.

In [4]:
type Sig_1 a = SigType a=>[[a]]

`blocks` is a functor level function that maps 0-sig to 1-sig with a given size of each element of 1-sig, so the definition is `blocks::n->Sig_0 a->Sig_1 a`. We require the size of each element must be power of 2. The same requirement goes to 0-sig.

In [5]:
blocks::Int->Sig_0 a->Sig_1 a
blocks n x
    | not $ ispower2 n = []
    | n<=0 = [x]
blocks _ [] = []
blocks n xs = take n xs:blocks n (drop n xs)

In [6]:
blocks 2 [0..7]

[[0.0,1.0],[2.0,3.0],[4.0,5.0],[6.0,7.0]]

In [7]:
fmap sum $ take 5 $ blocks 2 [0..]

[1.0,5.0,9.0,13.0,17.0]

### 3-sig

In [8]:
data TripleBlocks a = SigType a=>TripleBlocks{left::Sig_0 a,center::Sig_0 a,right::Sig_0 a}
type Sig_3 a = [TripleBlocks a]

In [9]:
instance SigType a => Show (TripleBlocks a) where
    show TripleBlocks{left=a,center=b,right=c} = "[" ++ show a ++ "," ++ show b ++ "," ++ show c ++ "]"

In [10]:
:t left

left :: forall a. TripleBlocks a -> Sig_0 a

In [11]:
foo=TripleBlocks{left=[1,2],center=[3,4],right=[5,6]}
print foo

[[1.0,2.0],[3.0,4.0],[5.0,6.0]]

In [12]:
print $ left foo
print $ center foo
print $ right foo

[1.0,2.0]

[3.0,4.0]

[5.0,6.0]

In [13]:
:t left

left :: forall a. TripleBlocks a -> Sig_0 a

In [14]:
foo = TripleBlocks{left=[1.0,2.0]}

In [15]:
print $ left foo

[1.0,2.0]

### Padding

In [16]:
data Extensions a = SigType a=>Extensions{extleft::Sig_0 a, extright::Sig_0 a}

instance SigType a=>Show (Extensions a) where
    show Extensions{extleft=l,extright=r} = "["++ show l ++ "," ++ show r ++ "]"

In [17]:
type GetExtensions a = SigType a=>Int->Sig_0 a->Extensions a

In [18]:
zeroPadding'::SigType a=>Int->Sig_0 a->TripleBlocks a
zeroPadding' n a = TripleBlocks{left=replicate n 0, center=a, right=replicate n 0}

In [19]:
zeroPadding::GetExtensions a
zeroPadding n a = Extensions{extleft=replicate n 0, extright=replicate n 0}

In [20]:
symetricPadding::GetExtensions a
symetricPadding n a = Extensions{extleft=lr,extright=rr} where
    lr = reverse $ take n a
    rr = take n $ reverse a

In [21]:
periodicPadding::GetExtensions a
periodicPadding n a = Extensions{extleft=lr,extright=rr} where
    lr = reverse $ take n  $ reverse a
    rr = take n a

#### Implementation using pattern matching

In [22]:
data PaddingMethod=ZERO|PERIODIC|SYMETRIC

In [23]:
pad'::SigType a=>PaddingMethod->Int->Sig_0 a->Extensions a
pad' ZERO = zeroPadding
pad' PERIODIC = periodicPadding
pad' SYMETRIC = symetricPadding

In [24]:
a=[1..8]
fmap (\x -> pad' x 4 a) [ZERO,PERIODIC,SYMETRIC]

[[[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0]],[[5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0]],[[4.0,3.0,2.0,1.0],[8.0,7.0,6.0,5.0]]]

### Interface implementation

In [25]:
class Padding f where
    pad::(SigType a)=>f->(Int->Sig_0 a-> Extensions a)

In [26]:
instance Padding PaddingMethod where
    pad ZERO = zeroPadding
    pad PERIODIC = periodicPadding
    pad SYMETRIC = symetricPadding

In [27]:
a=[1..8]
fmap (\x -> pad x 4 a) [ZERO,PERIODIC,SYMETRIC]

[[[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0]],[[5.0,6.0,7.0,8.0],[1.0,2.0,3.0,4.0]],[[4.0,3.0,2.0,1.0],[8.0,7.0,6.0,5.0]]]

#### Sliding windows --- windowed::Int->[a]->[[a]]

In [28]:
-- windowed on list, returning windows with n elements  each
windowed :: Int->[a]->[[a]]
windowed n x
    | n<=1 = [x]
windowed _ [] = []
windowed n (x:xs)
    | length(take n (x:xs))<n = [] -- `take n` makes sure it works on infinite list
    | otherwise = w:windowed n xs
    where w=take n (x:xs)
-- without guarantee each window has n element, then the following works:
-- windowed n x = Data.List.transpose $ take n $ tails x
-- or
-- windowed n x = map (tak n) $ tails x



In [29]:
take 5 $ windowed 3 [0..]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6]]

In [30]:
windowed 3 [0..7]

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7]]

In [31]:
oblocks::SigType a=>Sig_1 a->Sig_3 a
oblocks (x:xs)
    | length(take 3 (x:xs))<3 = []
    | otherwise = TripleBlocks{left=head w,center=w!!1,right=last w}:oblocks xs
    where w=take 3 (x:xs)

In [65]:
a=[1..8]
b=blocks 2 a
c=oblocks b
print b
print c

[[1.0,2.0],[3.0,4.0],[5.0,6.0],[7.0,8.0]]

[[[1.0,2.0],[3.0,4.0],[5.0,6.0]],[[3.0,4.0],[5.0,6.0],[7.0,8.0]]]

### Sig
The type class `Sig` federalizes the behavior of `fmap` by providing a new function `transform`, but it is too restrictive. What we intend to do here to restrict `transform` so that function `([a]->[b])` is permitted but function `([a]->b)` (e.g. `sum`) is not allowed, in other words we want to preserve the `Sig_1` structure. We don't know how to do this just yet, but it is the placeholder for future development on this issue.

In [32]:
class Sig s where
    transform::(a->a)->s a->s a

In [33]:
instance Sig [] where
    transform=fmap

In [34]:
a = [[1,2],[3,4]]
b = [[5,6],[7,8]]

In [35]:
a++b

[[1,2],[3,4],[5,6],[7,8]]

In [36]:
transform (take 2) $ a++b

[[1,2],[3,4],[5,6],[7,8]]

In [37]:
fmap sum $ a++b

[3,7,11,15]

In [38]:
foo = fmap (+) [1,2,3]
foo1 = pure (+)

In [39]:
:t foo1

foo1 :: forall {f :: * -> *} {a}. (Applicative f, Num a) => f (a -> a -> a)

In [40]:
pure (+) <*> [1,2,3] <*> [4,5,6]

Line 1: Use <$>
Found:
pure (+) <*> [1, 2, 3]
Why not:
((+) <$> [1, 2, 3])

[5,6,7,6,7,8,7,8,9]

In [41]:
[1..7] >>= (\x -> [[x-1, x, x+1]])

[[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7],[6,7,8]]

In [42]:
:t (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

In [43]:
import Control.Monad.Writer

sumN :: Int -> Writer String Int
sumN 0 = writer (0, "finish")
sumN n = do
    tell (show n ++ ",")
    s <- sumN (n-1)
    pure (n + s)

In [44]:
sumN 5

WriterT (Identity (15,"5,4,3,2,1,finish"))

In [45]:
pure (++[1,2]) <*> [[3,4],[5,6]]

Line 1: Use <$>
Found:
pure (++ [1, 2]) <*> [[3, 4], [5, 6]]
Why not:
(++ [1, 2]) <$> [[3, 4], [5, 6]]

[[3,4,1,2],[5,6,1,2]]

In [46]:
:t (<*>)

(<*>) :: forall (f :: * -> *) a b. Applicative f => f (a -> b) -> f a -> f b

In [47]:
:k ()

() :: *

In [48]:
{-
module Lax where

class Functor f => Closed f where
    (<*>) :: f (a -> b) -> f a -> f b
    unit :: f ()

module Monoid where

class Functor f => Monoidal f where
    (>*<) :: (f a, f b) -> f (a, b)
    unit :: f ()
-}


## Decimation

In [49]:
decimation::Int->[a]->[a]
decimation n x
    | n<2 = x
decimation _ [] = []
decimation n (x:xs) = x:decimation n (drop (n-1) xs)

In [50]:
decimation 2 [1,2,3]

[1,3]

In [51]:
take 5 $ decimation 2 [0..]

[0,2,4,6,8]

In [52]:
decimation 1 [1..5]

[1,2,3,4,5]

## Translation

## Utils
### Linespace

```{note}
Integral literals written using scientific notation will be desugared using fromInteger, whereas any literals which aren’t integral will be desugared using fromRational as usual.
```

In [53]:
linespace' :: SigType a=>a->a->Int->[a]
linespace' _ _ n
    | n<=1 = []
linespace' a1 a2 _
    | a2<=a1 = []
linespace' a1 a2 n = map (\ i -> a1+dt*fromIntegral i) [0..(n-1)]
    where
        dt=(a2-a1)/fromIntegral(n-1)
--linespace a1 a2 n = [let dt=(a2-a1)/(n-1) in a1+i*dt | i<-[0..(n-1)]]

```{warning}
The fromIntegral function is actually unsafe and should be used
with care. The problem is that this function doesn’t check for underflows and
overflows in data. This could potentially lead to corrupting data when doing
incompatible type conversion.
```

In [54]:
linespace :: (SigType a, Integral n)=>a->a->n->Sig_0 a
linespace a1 a2 _
    | a2<=a1 = []
linespace a b n
    | n<=0 = []
    | n==1 = [a, b]
linespace a b n = [let dt=(b-a)/fromIntegral(n-1) in a+dt*fromIntegral i | i<-[0..(n-1)]]

In [55]:
abs <$> linespace (-1) 1 8

[1.0,0.7142857142857143,0.4285714285714286,0.1428571428571429,0.1428571428571428,0.4285714285714284,0.7142857142857142,1.0]

In [56]:
fmap sum $ blocks 2 $ linespace (-1) 1 8

[-1.7142857142857144,-0.5714285714285715,0.5714285714285712,1.7142857142857142]

In [57]:
linespace 1.0 (-1) 1

[]

In [58]:
len::[a]->Int
len [] = 0
len xs = foldr (\ x -> (+) 1) 0 xs

In [59]:
len [0..7]

8

In [60]:
foo=len [0..]

### Binary Tree

In [61]:
data Tree a = EmptyTree | Node a (Tree a) (Tree a) deriving (Show, Read, Eq) 

## Visualization

In [62]:
:set -XExtendedDefaultRules
import Graphics.Matplotlib

: 

In [ ]:
import Graphics.Rendering.Chart.Backend.Diagrams

: 

In [ ]:
import Diagrams.TwoD.Apollonian

: 